# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [173]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv') # открываю файл с данными
df.info() # смотрю информацию по файлу

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [174]:
df.head(20) #изучаю датафрейм

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


>**Вывод:**  
Всего в таблице 12 столбцов и 21525 строк.  
В столбцах **days_employed** и **total_income** есть пропущенные значения, помимо того в стобце **days_employed** есть отрицательные значения и формат данных float


## Шаг 2. Предобработка данных

### Обработка пропусков

In [175]:
df["days_employed"].isnull().sum() #считаю количество пропусков в столбце общий трудовой стаж работы

2174

In [176]:
df["total_income"].isnull().sum() #считаю количество пропусков в столбце ежемесячный доход

2174

In [177]:
#df[df.gender == "XNA"]
df.loc[df.gender == "XNA", "gender"] = "F" #меняю значение непонятного пола на Female

In [178]:
df["gender"].unique() #смотрю результат

array(['F', 'M'], dtype=object)

In [131]:
median_dob_years = df["dob_years"].median
#заменяю значения возраста меньше 18 на медианные, так как кредиты можно брать с 18 лет
df.loc[df.dob_years < 18, "dob_years"] = median_dob_years

In [179]:
df_days_employed_median = df.groupby("income_type")["days_employed"].median().reset_index()
# создал новый ДФ с медианными значениями стажа работы по группам занятости
df_days_employed_median

,income_type,days_employed
0,безработный,366413.652744
1,в декрете,-3296.759962
2,госслужащий,-2689.368353
3,компаньон,-1547.382223
4,пенсионер,365213.306266
5,предприниматель,-520.848083
6,сотрудник,-1574.202821
7,студент,-578.751554


In [180]:
df_total_income_median = df.groupby("income_type")["total_income"].median().reset_index()
# создал новый ДФ с медианными значениями средней зарплаты по группам занятости
df_total_income_median

,income_type,total_income
0,безработный,131339.751676
1,в декрете,53829.130729
2,госслужащий,150447.935283
3,компаньон,172357.950966
4,пенсионер,118514.486412
5,предприниматель,499163.144947
6,сотрудник,142594.396847
7,студент,98201.625314


In [181]:
df_unemployed = df[df.income_type == "безработный"] #создаю ДФ по категории безработный, тут пропусков нет
df_unemployed

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
14798,0,395302.838654,45,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью


In [182]:
df_decree = df[df.income_type == "в декрете"] #создаю ДФ по категории в декрете, тут пропусков нет
df_decree

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20845,2,-3296.759962,39,СРЕДНЕЕ,1,женат / замужем,0,F,в декрете,1,53829.130729,автомобиль


In [183]:
df_civil_servante = df[df.income_type == "госслужащий"] 
#создаю ДФ по категории госслужащий, меняю пропуски на медианные значения
df_civil_servante["days_employed"] = df_civil_servante["days_employed"].fillna(df_days_employed_median.loc[2, "days_employed"])
df_civil_servante["total_income"] = df_civil_servante["total_income"].fillna(df_total_income_median.loc[2, "total_income"])
df_civil_servante.isnull().sum()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [184]:
df_companion = df[df.income_type == "компаньон"]
#создаю ДФ по категории компаньон, меняю пропуски на медианные значения
df_companion["days_employed"] = df_companion["days_employed"].fillna(df_days_employed_median.loc[3, "days_employed"])
df_companion["total_income"] = df_companion["total_income"].fillna(df_total_income_median.loc[3, "total_income"])
df_companion.isnull().sum()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [185]:
df_retiree = df[df.income_type == "пенсионер"]
#создаю ДФ по категории пенсионер, меняю пропуски на медианные значения
df_retiree["days_employed"] = df_retiree["days_employed"].fillna(df_days_employed_median.loc[4, "days_employed"])
df_retiree["total_income"] = df_retiree["total_income"].fillna(df_total_income_median.loc[4, "total_income"])
df_retiree.isnull().sum()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [186]:
df_boss = df[df.income_type == "предприниматель"]
#создаю ДФ по категории предприниматель, меняю пропуски на медианные значения
df_boss["days_employed"] = df_boss["days_employed"].fillna(df_days_employed_median.loc[5, "days_employed"])
df_boss["total_income"] = df_boss["total_income"].fillna(df_total_income_median.loc[5, "total_income"])
df_boss.isnull().sum()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [187]:
df_employee = df[df.income_type == "сотрудник"]
#создаю ДФ по категории сотрудник, меняю пропуски на медианные значения
df_employee["days_employed"] = df_employee["days_employed"].fillna(df_days_employed_median.loc[6, "days_employed"])
df_employee["total_income"] = df_employee["total_income"].fillna(df_total_income_median.loc[6, "total_income"])
df_employee.isnull().sum()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [188]:
df_student = df[df.income_type == "студент"] #создаю ДФ по категории студент, тут пропусков нет
df_student.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [189]:
df = pd.concat([df_unemployed, df_decree, df_civil_servante, df_companion, df_retiree, df_boss, \
               df_employee, df_student], ignore_index=True)
#объединяю полученные ДФ в один смотрю результат
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,337524.466835,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
1,0,395302.838654,45,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью
2,2,-3296.759962,39,СРЕДНЕЕ,1,женат / замужем,0,F,в декрете,1,53829.130729,автомобиль
3,0,-2689.368353,41,среднее,1,женат / замужем,0,M,госслужащий,0,150447.935283,образование
4,0,-2689.368353,50,среднее,1,женат / замужем,0,F,госслужащий,0,150447.935283,сделка с подержанным автомобилем
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21521,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21522,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21523,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [190]:
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

>**Вывод**  
Пропущенных значения в столбцах **days_employed** и **total_income** одинаково в результате полученного ДФ видно что количество строк **равно** количеству пропущенных значений это означает что значения пропущенны **в одних и тех же строках.** Клиенты которые не давали информацию о ежемесячном доходе также не давали информацию о стаже работы.  
Пропущенные значения имеют колличественный тип и были заменены на медианные значения

### Замена типа данных

In [191]:
df["days_employed"] = df["days_employed"].astype("int") #меняю тип данных на int так как дни целые числа 
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,337524,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
1,0,395302,45,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью
2,2,-3296,39,СРЕДНЕЕ,1,женат / замужем,0,F,в декрете,1,53829.130729,автомобиль
3,0,-2689,41,среднее,1,женат / замужем,0,M,госслужащий,0,150447.935283,образование
4,0,-2689,50,среднее,1,женат / замужем,0,F,госслужащий,0,150447.935283,сделка с подержанным автомобилем
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-2351,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21521,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21522,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21523,2,-1984,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [192]:
df["total_income"] = df["total_income"].astype("int") #меняю тип данных на int   
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,337524,31,среднее,1,женат / замужем,0,M,безработный,1,59956,покупка жилья для сдачи
1,0,395302,45,Высшее,0,гражданский брак,1,F,безработный,0,202722,ремонт жилью
2,2,-3296,39,СРЕДНЕЕ,1,женат / замужем,0,F,в декрете,1,53829,автомобиль
3,0,-2689,41,среднее,1,женат / замужем,0,M,госслужащий,0,150447,образование
4,0,-2689,50,среднее,1,женат / замужем,0,F,госслужащий,0,150447,сделка с подержанным автомобилем
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-2351,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949,покупка коммерческой недвижимости
21521,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21522,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля
21523,2,-1984,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047,на покупку автомобиля


>**Вывод**  
Поменял формат стобцов **days_employed**, **total_income** на целочисленный , использовал метод astype()  

### Обработка дубликатов

In [193]:
df["education"] = df["education"].str.lower() #перевожу значение столбца к нижнему регистру
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,337524,31,среднее,1,женат / замужем,0,M,безработный,1,59956,покупка жилья для сдачи
1,0,395302,45,высшее,0,гражданский брак,1,F,безработный,0,202722,ремонт жилью
2,2,-3296,39,среднее,1,женат / замужем,0,F,в декрете,1,53829,автомобиль
3,0,-2689,41,среднее,1,женат / замужем,0,M,госслужащий,0,150447,образование
4,0,-2689,50,среднее,1,женат / замужем,0,F,госслужащий,0,150447,сделка с подержанным автомобилем
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-2351,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949,покупка коммерческой недвижимости
21521,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21522,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля
21523,2,-1984,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047,на покупку автомобиля


In [194]:
df.duplicated().sum() #ищу дубликаты во всем ДФ

71

In [196]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

>**Вывод**:  
В указанном ДФ **71** дубрипующая строка, это вызванно тем что *я произвел замену отсутствующих значений*

### Лемматизация

In [197]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

text = df["purpose"].unique()#получаю уникальные значения в столбце для лемматизации

lemmas = [] #создаю пустой список
#каждую строчку в 'text' лемматизиру., получаем список лемм и добавляем значения в пустой список 'lemmas'
for i in text:
    lemma = m.lemmatize(i)
    lemmas.append(lemma)
lemmas

[['покупка', ' ', 'жилье', ' ', 'для', ' ', 'сдача', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['автомобиль', '\n'],
 ['образование', '\n'],
 ['сделка', ' ', 'с', ' ', 'подержанный', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['получение', ' ', 'высокий', ' ', 'образование', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'жилье', '\n'],
 ['покупка', ' ', 'свой', ' ', 'жилье', '\n'],
 ['операция', ' ', 'с', ' ', 'недвижимость', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['жилье', '\n'],
 ['профильный', ' ', 'образование', '\n'],
 ['заниматься', ' ', 'высокий', ' ', 'образование', '\n'],
 ['сделка', ' ', 'с', ' ', 'автомобиль

>**Вывод**:  
Чтобы провести **лемматизацию** выделил уникальные значения в столбце **purpose** и через цикл лемматизировакл каждое из них которое добавил в отдельный список

### Категоризация данных

In [ ]:
def purpose_func(purpose):
    """ Функция которая распределяет по категориям цели для которых люди обращаются за кредитом"""
    lemmas_row = m.lemmatize(purpose)
    for i in lemmas:
        if 'авто' in i:
            return 'автомобиль'
        if 'недвиж' or 'жил' in i:
            return  'недвижимость'
        if 'свад' in i:
            return  'свадба'
        if 'образов' in i:
            return 'образование'

In [ ]:
df['purpose_lemm'] = df['purpose'].apply(purpose_func) 
#создаю новый столбец где сохраню категории (цели) для которых берут кредиты и к столбцу с целями кредита применил функцию
df.head()

In [ ]:
df['purpose_lemm'].value_counts()

In [198]:
# КОД РЕВЬЮВЕРА

# чуть подправил твою функцию

def purpose_func_new(row):
    """ Функция которая распределяет по категориям цели для которых люди обращаются за кредитом"""
    lemmas_row = m.lemmatize(row['purpose'])
    
    try:
        
        if 'автомобиль' in lemmas_row:
            return 'операции с автомобилем'
        if ('жилье' in lemmas_row) or ('недвижимость' in lemmas_row ):
            return 'операции с недвижимостью'
        if 'свадьба' in lemmas_row:
            return 'проведение свадьбы'
        if 'образование' in lemmas_row:
            return 'получение образования'
        
    except:
        
        return 'нет категории'
    
df['new_purpose'] = df.apply(purpose_func_new, axis=1)

In [199]:
# КОД РЕВЬЮВЕРА

df['new_purpose'].value_counts() # уже лучше, смотри ка!

# далее в блоке ответов на вопросы работай с этими целями

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: new_purpose, dtype: int64

In [200]:
def func_income(row):
    try:
        income = row["total_income"]
        if income <= 50000:
            return "малый доход"
        if income >= 50000 and income <= 100000:
            return "средний доход"
        if income >= 100001 and income <= 200000:
            return "высокий доход"
        if income >= 200001 :
            return "очень высокий доход"
    except:
        print("Ты допустил ошибку")

In [201]:
df["income"] = df.apply(func_income, axis=1)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_purpose,income
0,1,337524,31,среднее,1,женат / замужем,0,M,безработный,1,59956,покупка жилья для сдачи,операции с недвижимостью,средний доход
1,0,395302,45,высшее,0,гражданский брак,1,F,безработный,0,202722,ремонт жилью,операции с недвижимостью,очень высокий доход
2,2,-3296,39,среднее,1,женат / замужем,0,F,в декрете,1,53829,автомобиль,операции с автомобилем,средний доход
3,0,-2689,41,среднее,1,женат / замужем,0,M,госслужащий,0,150447,образование,получение образования,высокий доход
4,0,-2689,50,среднее,1,женат / замужем,0,F,госслужащий,0,150447,сделка с подержанным автомобилем,операции с автомобилем,высокий доход


In [202]:
# КОД РЕВЬЮВЕРА

df["income"].value_counts()

высокий доход          11924
очень высокий доход     5067
средний доход           4091
малый доход              372
Name: income, dtype: int64

In [203]:
df['total_income_group'] = pd.qcut(df['total_income'], 4,  labels = ['низкий', 'средний', 'выше среднего', 'высокий'])
df.head() #воспользовался советом

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_purpose,income,total_income_group
0,1,337524,31,среднее,1,женат / замужем,0,M,безработный,1,59956,покупка жилья для сдачи,операции с недвижимостью,средний доход,низкий
1,0,395302,45,высшее,0,гражданский брак,1,F,безработный,0,202722,ремонт жилью,операции с недвижимостью,очень высокий доход,высокий
2,2,-3296,39,среднее,1,женат / замужем,0,F,в декрете,1,53829,автомобиль,операции с автомобилем,средний доход,низкий
3,0,-2689,41,среднее,1,женат / замужем,0,M,госслужащий,0,150447,образование,получение образования,высокий доход,выше среднего
4,0,-2689,50,среднее,1,женат / замужем,0,F,госслужащий,0,150447,сделка с подержанным автомобилем,операции с автомобилем,высокий доход,выше среднего


In [211]:
def func_child(row):
    try:
        child = row["children"]
        if child == 0:
            return "без детей"
        if child > 0 and child <= 2:
            return "1-2 ребенка"
        if child > 2:
            return "многодетная"
    except:
        print("Ты допустил ошибку")

In [212]:
df["children_group"] = df.apply(func_child, axis=1)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_purpose,income,total_income_group,children_group
0,1,337524,31,среднее,1,женат / замужем,0,M,безработный,1,59956,покупка жилья для сдачи,операции с недвижимостью,средний доход,низкий,1-2 ребенка
1,0,395302,45,высшее,0,гражданский брак,1,F,безработный,0,202722,ремонт жилью,операции с недвижимостью,очень высокий доход,высокий,без детей
2,2,-3296,39,среднее,1,женат / замужем,0,F,в декрете,1,53829,автомобиль,операции с автомобилем,средний доход,низкий,1-2 ребенка
3,0,-2689,41,среднее,1,женат / замужем,0,M,госслужащий,0,150447,образование,получение образования,высокий доход,выше среднего,без детей
4,0,-2689,50,среднее,1,женат / замужем,0,F,госслужащий,0,150447,сделка с подержанным автомобилем,операции с автомобилем,высокий доход,выше среднего,без детей


>**Вывод**:  
Данные категорированны по **целям кредита** для которых люди идут в банк, автомобили, недвижимость, образование, проведение свадеб.   
Выбрал категорировать именно по этому признаку потому что менеджеры в банке заполняя столбец *цели кредита* указывают все со слов клиента данных получается много и с такими данными тяжело работать  
*В течении часа пытался прочитать функцию, но видно данных очень много поэтому не прогрузил(( сорри*  
**Еще сделал категорирование данных по доходу чтобы ответить на вопрос** и **по наличию детей**

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [204]:
df["children"] = df["children"].abs()# в данных есть значение -1 ребенок, предположу что это ошибка оператора при вводе данных
# меняю отрицательное число на положительное

In [213]:
df_child = df[df.children < 20] #отбрасываю значения с 20 детьми
df_child

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_purpose,income,total_income_group,children_group
0,1,337524,31,среднее,1,женат / замужем,0,M,безработный,1,59956,покупка жилья для сдачи,операции с недвижимостью,средний доход,низкий,1-2 ребенка
1,0,395302,45,высшее,0,гражданский брак,1,F,безработный,0,202722,ремонт жилью,операции с недвижимостью,очень высокий доход,высокий,без детей
2,2,-3296,39,среднее,1,женат / замужем,0,F,в декрете,1,53829,автомобиль,операции с автомобилем,средний доход,низкий,1-2 ребенка
3,0,-2689,41,среднее,1,женат / замужем,0,M,госслужащий,0,150447,образование,получение образования,высокий доход,выше среднего,без детей
4,0,-2689,50,среднее,1,женат / замужем,0,F,госслужащий,0,150447,сделка с подержанным автомобилем,операции с автомобилем,высокий доход,выше среднего,без детей
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-2351,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949,покупка коммерческой недвижимости,операции с недвижимостью,высокий доход,средний,1-2 ребенка
21450,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,операции с недвижимостью,средний доход,низкий,1-2 ребенка
21451,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,операции с автомобилем,очень высокий доход,высокий,многодетная
21452,2,-1984,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047,на покупку автомобиля,операции с автомобилем,средний доход,низкий,1-2 ребенка


In [235]:
df_child_group = df_child.groupby("children_group").agg({"debt":"sum", "family_status_id":"count"}).reset_index()
df_child_group
#группирую значения по категории наличия детей и вычисляю сколько раз просрочили кредит 
#а также сколько всего людей в данной категории

,children_group,debt,family_status_id
0,1-2 ребенка,639,6907
1,без детей,1063,14091
2,многодетная,31,380


In [237]:
df_child_group["%_no_return"] = df_child_group["debt"] / df_child_group["family_status_id"] * 100
df_child_group

,children_group,debt,family_status_id,%_no_return
0,1-2 ребенка,639,6907,9.251484
1,без детей,1063,14091,7.543822
2,многодетная,31,380,8.157895


>**Вывод**:  
Такая зависимость **наблюдается**, люди у которых есть дети чаще не возвращают кредиты чем те у которых детей нет


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [238]:
df_family = df.groupby("family_status").agg({"debt":"sum", "family_status_id":"count"}).reset_index()
df_family

,family_status,debt,family_status_id
0,Не женат / не замужем,274,2810
1,в разводе,85,1195
2,вдовец / вдова,63,959
3,гражданский брак,388,4151
4,женат / замужем,931,12339


In [239]:
df_family["%_no_return"] = df_family["debt"] / df_family["family_status_id"] * 100
df_family

,family_status,debt,family_status_id,%_no_return
0,Не женат / не замужем,274,2810,9.750890
1,в разводе,85,1195,7.112971
2,вдовец / вдова,63,959,6.569343
3,гражданский брак,388,4151,9.347145
4,женат / замужем,931,12339,7.545182


>**Вывод:**  
Люди **не женатые** (и находящиеся в гражданском браке) чаще имели задолженность по возврату кредитов, вместе с тем люди потерявшие близких кредиты стараются возвращать вовремя

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [240]:
df_income = df.groupby("income").agg({"debt":"sum", "family_status_id":"count"}).reset_index()
df_income

,income,debt,family_status_id
0,высокий доход,1029,11924
1,малый доход,23,372
2,очень высокий доход,358,5067
3,средний доход,331,4091


In [241]:
df_income["%_no_return"] = df_income["debt"] / df_income["family_status_id"] * 100
df_income

,income,debt,family_status_id,%_no_return
0,высокий доход,1029,11924,8.629654
1,малый доход,23,372,6.182796
2,очень высокий доход,358,5067,7.065325
3,средний доход,331,4091,8.090931


>**Вывод:**  
*Сам удивлен*  
Как ни странно но зависимость между доходом людей и невозвратом кредита в срок есть, чем выше люди зарабатывают тем чаще не возвращают кредит вовремя, вместе с тем люди имеющие малый доход *менее 50 000* практически всегда возвращают кредиты вовремя

- Как разные цели кредита влияют на его возврат в срок?

In [243]:
df_purpose = df.groupby("new_purpose").agg({"debt":"sum", "family_status_id":"count"}).reset_index()
df_purpose

,new_purpose,debt,family_status_id
0,операции с автомобилем,403,4306
1,операции с недвижимостью,782,10811
2,получение образования,370,4013
3,проведение свадьбы,186,2324


In [244]:
df_purpose["%_no_return"] = df_purpose["debt"] / df_purpose["family_status_id"] * 100
df_purpose

,new_purpose,debt,family_status_id,%_no_return
0,операции с автомобилем,403,4306,9.359034
1,операции с недвижимостью,782,10811,7.233373
2,получение образования,370,4013,9.220035
3,проведение свадьбы,186,2324,8.003442


>**Вывод:**  
Люди берущие кредиты на **операции с автомобилями и получение образования** чаще всего не возвращают кредиты в срок, вместе с тем люди берущие кредиты на **нежвижимость** возвращают во время

## Шаг 4. Общий вывод

>При наличии детей % просрочки выше чем у пар не имющих детей.  
Влияние семейного положения имеет схожий характер больше всего просрочек у тех кто не женат или живет в гражданском браке.  
У тех у кого доход высокий и процент не возврата кредитов остается на высоком уровне
Из целей кредита самой надежной является ипотека или сделки с нежвижимостью